In [5]:
#import beautifulsoup
from bs4 import BeautifulSoup

In [6]:
#fetch html contents
import requests
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, 'html.parser')
#print(soup.prettify())

In [7]:
#retrieve the table
address_table = soup.find("table", attrs={"class": "wikitable"})
address_table_data = address_table.tbody.find_all("tr")
#address_table_data

In [8]:
#retrieve the headers
#address_table_data[0]
headers_data = address_table_data[0].find_all("th")
headers_data
headers = []
for header in headers_data:
    headers.append(header.text.replace('\n', ' ').strip())

print(headers)

['Postal Code', 'Borough', 'Neighborhood']


In [9]:
#data frame in panda
import pandas as pd
import numpy as np
# Get all the rows of table
table_data = []
for tr in address_table_data: # find all tr's from table's tbody
    #print(tr)
    cols = tr.find_all('td')
    cols = [td.text.replace('\n', '').strip() for td in cols]
    table_data.append([td for td in cols if td])

#table_data
df = pd.DataFrame(table_data, columns = headers)
print(df)


    Postal Code       Borough  \
0          None          None   
1           M1A  Not assigned   
2           M2A  Not assigned   
3           M3A    North York   
4           M4A    North York   
..          ...           ...   
176         M5Z  Not assigned   
177         M6Z  Not assigned   
178         M7Z  Not assigned   
179         M8Z     Etobicoke   
180         M9Z  Not assigned   

                                          Neighborhood  
0                                                 None  
1                                                 None  
2                                                 None  
3                                            Parkwoods  
4                                     Victoria Village  
..                                                 ...  
176                                               None  
177                                               None  
178                                               None  
179  Mimico NW, The Queensway Wes

In [10]:
# get geolocation data
import geocoder # import geocoder

In [11]:
# initialize your variable to None
lat_lng_coords = None

In [12]:
# Get geolocation from the CSV file
df_geoloc = pd.read_csv("C:\\NecessaryFolders\\PYTHON_IBM_DATA_SCIENCE\\notebooks\\week3_assignment_submission\\Geospatial_Coordinates.csv")
df_geoloc.head()
df_geoloc.shape


(103, 3)

In [13]:
# Assign latitude and longitude in df
df["Latitude"] = ""
df["Longitude"] = ""
for i in range(len(df)):
    for j in range(len(df_geoloc)):
        if df['Postal Code'].iloc[i] == df_geoloc['Postal Code'].iloc[j]:
            df['Latitude'].iloc[i] = df_geoloc['Latitude'].iloc[j]
            df['Longitude'].iloc[i] = df_geoloc['Longitude'].iloc[j]
            #print(df_geoloc['Latitude'].iloc[j])

df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,None,None,None,,
1,M1A,Not assigned,None,,
2,M2A,Not assigned,None,,
3,M3A,North York,Parkwoods,43.7533,-79.3297
4,M4A,North York,Victoria Village,43.7259,-79.3156
...,...,...,...,...,...
176,M5Z,Not assigned,None,,
177,M6Z,Not assigned,None,,
178,M7Z,Not assigned,None,,
179,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.6288,-79.521


In [14]:
# filter data
# drop cells with 'Not assigned' in Borough
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df['Neighborhood'].replace('None', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df.shape

(103, 5)

In [15]:
# view updated dataframe
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.7533,-79.3297
4,M4A,North York,Victoria Village,43.7259,-79.3156
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
...,...,...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,-79.5069
166,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
169,M7Y,East Toronto,Business reply mail Processing Centre,43.6627,-79.3216
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6363,-79.4985


In [16]:
# import different libraries
import random # library for random number generation
import folium # map rendering library
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

#from sklearn.cluster import KMeans 
#from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


In [17]:
# retrieve longitude and latitude of Toronto city
from geopy.geocoders import Nominatim

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [18]:
# see df
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.7533,-79.3297
4,M4A,North York,Victoria Village,43.7259,-79.3156
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
...,...,...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,-79.5069
166,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
169,M7Y,East Toronto,Business reply mail Processing Centre,43.6627,-79.3216
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6363,-79.4985


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
# initialize credentials for foursquare
CLIENT_ID = 'P0CICFIWIUCIGNJ3INWISZ4YWZJGXT12DLAIAWJRKD0V0KOE' # your Foursquare ID
CLIENT_SECRET = 'OOVTJWQ42URHTTNFI334C1FMUNZKHSSDI4AY4WZ15MXRBXPG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P0CICFIWIUCIGNJ3INWISZ4YWZJGXT12DLAIAWJRKD0V0KOE
CLIENT_SECRET:OOVTJWQ42URHTTNFI334C1FMUNZKHSSDI4AY4WZ15MXRBXPG


In [30]:
# define a function to retrieve venues for all neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            2000)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# get all the venues in Toronto
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [32]:
# check the data
print(toronto_venues.shape)
toronto_venues.head()

(2110, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Wexford, Maryvale",4,4,4,4,4,4
Willowdale,39,39,39,39,39,39
Woburn,4,4,4,4,4,4


In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# examine the new dataframe size
toronto_onehot.shape

(2110, 272)

In [40]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.047619,0.000000,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,"Wexford, Maryvale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
89,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.025641,0.0,0.0,0.0,0.0
90,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
91,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [41]:
#new size
toronto_grouped.shape

(93, 272)

In [42]:
# print each neighborhood along with the top 10 most common venues
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                             venue  freq
0                           Lounge   0.2
1                   Breakfast Spot   0.2
2                     Skating Rink   0.2
3        Latin American Restaurant   0.2
4                   Clothing Store   0.2
5                      Yoga Studio   0.0
6        Middle Eastern Restaurant   0.0
7  Molecular Gastronomy Restaurant   0.0
8       Modern European Restaurant   0.0
9                Mobile Phone Shop   0.0


----Alderwood, Long Branch----
                 venue  freq
0          Pizza Place  0.29
1          Coffee Shop  0.14
2       Sandwich Place  0.14
3                  Pub  0.14
4             Pharmacy  0.14
5                  Gym  0.14
6                 Park  0.00
7  Monument / Landmark  0.00
8    Martial Arts Dojo  0.00
9       Massage Studio  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2               

                 venue  freq
0          Coffee Shop  0.13
1             Aquarium  0.05
2                Hotel  0.04
3                 Café  0.04
4              Brewery  0.03
5  Sporting Goods Shop  0.03
6           Restaurant  0.03
7   Italian Restaurant  0.03
8  Fried Chicken Joint  0.03
9       Scenic Lookout  0.03


----High Park, The Junction South----
                       venue  freq
0         Mexican Restaurant  0.09
1                       Café  0.09
2            Thai Restaurant  0.09
3                  Speakeasy  0.04
4        Arts & Crafts Store  0.04
5       Fast Food Restaurant  0.04
6                  Bookstore  0.04
7        Fried Chicken Joint  0.04
8  Cajun / Creole Restaurant  0.04
9     Furniture / Home Store  0.04


----Hillcrest Village----
                        venue  freq
0        Fast Food Restaurant   0.2
1                     Dog Run   0.2
2                        Pool   0.2
3    Mediterranean Restaurant   0.2
4                 Golf Course   0.2
5          M

                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.06
2         Cocktail Bar  0.05
3            Gastropub  0.04
4  American Restaurant  0.04
5              Theater  0.03
6   Seafood Restaurant  0.03
7       Clothing Store  0.03
8  Moroccan Restaurant  0.03
9           Restaurant  0.03


----St. James Town, Cabbagetown----
                venue  freq
0         Pizza Place  0.07
1         Coffee Shop  0.07
2          Restaurant  0.07
3  Italian Restaurant  0.05
4              Bakery  0.05
5                 Pub  0.05
6                Café  0.05
7         Snack Place  0.02
8        Liquor Store  0.02
9              Market  0.02


----Steeles West, L'Amoreaux West----
                  venue  freq
0  Fast Food Restaurant  0.14
1    Chinese Restaurant  0.14
2              Pharmacy  0.07
3           Coffee Shop  0.07
4           Pizza Place  0.07
5     Electronics Store  0.07
6        Sandwich Place  0.07
7                  Bank  0.07
8          Camera Store  

In [43]:
# function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
# top 10 venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Clothing Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Gym,Pharmacy,Coffee Shop,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Supermarket,Ice Cream Shop,Sushi Restaurant,Shopping Mall,Restaurant,Deli / Bodega,Fried Chicken Joint
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Thai Restaurant,Sandwich Place,Italian Restaurant,Liquor Store,Juice Bar,Restaurant,Butcher,Café,Indian Restaurant


In [101]:
# import k-means from clustering stage
import random
from sklearn.cluster import KMeans
# Run k-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
k_means = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
k_means.labels_[0:] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 3, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 2])

In [102]:
# new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
df.dropna()
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged.head() # check the last columns!

In [103]:
toronto_merged.shape
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M3A,North York,Parkwoods,43.7533,-79.3297,1.0,Bus Stop,Park,Construction & Landscaping,Food & Drink Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,M4A,North York,Victoria Village,43.7259,-79.3156,1.0,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Farmers Market,Event Space
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Vietnamese Restaurant,Event Space,Miscellaneous Shop,Coffee Shop,Distribution Center,Dim Sum Restaurant
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,-79.5069,0.0,Park,River,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
166,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,1.0,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Yoga Studio,Pizza Place,Pub,Café,Hotel,Mediterranean Restaurant
169,M7Y,East Toronto,Business reply mail Processing Centre,43.6627,-79.3216,1.0,Light Rail Station,Garden Center,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6363,-79.4985,4.0,Deli / Bodega,Baseball Field,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Farmers Market


In [104]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [105]:
#toronto_merged.dropna()
toronto_merged.dropna(subset = ["Cluster Labels"], inplace=True)
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M3A,North York,Parkwoods,43.7533,-79.3297,1.0,Bus Stop,Park,Construction & Landscaping,Food & Drink Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,M4A,North York,Victoria Village,43.7259,-79.3156,1.0,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Farmers Market,Event Space
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Vietnamese Restaurant,Event Space,Miscellaneous Shop,Coffee Shop,Distribution Center,Dim Sum Restaurant
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,-79.5069,0.0,Park,River,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
166,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,1.0,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Yoga Studio,Pizza Place,Pub,Café,Hotel,Mediterranean Restaurant
169,M7Y,East Toronto,Business reply mail Processing Centre,43.6627,-79.3216,1.0,Light Rail Station,Garden Center,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6363,-79.4985,4.0,Deli / Bodega,Baseball Field,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Farmers Market


In [106]:
#visualize the cluster

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [107]:
# cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,York,0.0,Park,Women's Store,Bar,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
55,Scarborough,0.0,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
130,Central Toronto,0.0,Park,Summer Camp,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
136,Scarborough,0.0,Park,Playground,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
148,Downtown Toronto,0.0,Park,Playground,Trail,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dance Studio
161,Etobicoke,0.0,Park,River,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [108]:
# cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,1.0,Bus Stop,Park,Construction & Landscaping,Food & Drink Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,North York,1.0,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
5,Downtown Toronto,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Farmers Market,Event Space
6,North York,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Vietnamese Restaurant,Event Space,Miscellaneous Shop,Coffee Shop,Distribution Center,Dim Sum Restaurant
7,Downtown Toronto,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
...,...,...,...,...,...,...,...,...,...,...,...,...
157,Downtown Toronto,1.0,Restaurant,Coffee Shop,Pizza Place,Pub,Bakery,Italian Restaurant,Café,Beer Store,Snack Place,Japanese Restaurant
158,Downtown Toronto,1.0,Coffee Shop,Café,Japanese Restaurant,Restaurant,Hotel,Gym,Deli / Bodega,Salad Place,Seafood Restaurant,Steakhouse
166,Downtown Toronto,1.0,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Yoga Studio,Pizza Place,Pub,Café,Hotel,Mediterranean Restaurant
169,East Toronto,1.0,Light Rail Station,Garden Center,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop


In [109]:
# cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,East York,2.0,Convenience Store,Park,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
96,York,2.0,Convenience Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store,Deli / Bodega
99,York,2.0,Convenience Store,Park,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
101,North York,2.0,Park,Bank,Convenience Store,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [110]:
# cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,York,3.0,Trail,Hockey Arena,Field,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store


In [111]:
# cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,North York,4.0,Baseball Field,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Farmers Market
170,Etobicoke,4.0,Deli / Bodega,Baseball Field,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Farmers Market
